UI to work with SBI stuff

In [1]:
import os
import subprocess

def source_lmod_script(script_path):
    """
    Source an Lmod/module script and import environment variables into Python safely,
    suppressing terminal warnings.
    """
    # Use a non-interactive login shell (bash -l), redirect errors
    command = f'bash -l -c "source {script_path} >/dev/null 2>&1; printenv -0"'
    
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    out, _ = proc.communicate()
    
    # Parse null-separated environment variables
    for env_var in out.split(b'\0'):
        if env_var:
            key, _, value = env_var.partition(b'=')
            os.environ[key.decode()] = value.decode()

# Example usage
M3_BUILD_DIR = "/home/henryi/scratch/venvs/.venv_sbi/bin/"
TUTORIAL_BUILD_DIR = M3_BUILD_DIR
source_lmod_script(f"{M3_BUILD_DIR}/setup.MaCh3.sh")
source_lmod_script(f"{TUTORIAL_BUILD_DIR}/setup.MaCh3Tutorial.sh")
os.environ["OMP_NUM_THREADS"] = "8"


In [2]:
from mach3sbitools.ui.sbi_ui import MaCh3SbiUI
from mach3sbitools.utils.device_handler import TorchDeviceHander
from mach3sbitools.sbi.mach3_prior import MaCh3TorchPrior

from pathlib import Path

from sbi.utils import BoxUniform

In [3]:
input_file = Path("/home/henryi/sft/MaCh3Tutorial/TutorialConfigs/FitterConfig.yaml")
ui = MaCh3SbiUI(str(input_file))
device_handler = TorchDeviceHander()

[Monitor.cpp][info] ##################################
[Monitor.cpp][info] Welcome to:  
[Monitor.cpp][info]   __  __        _____ _     ____  
[Monitor.cpp][info]  |  \/  |      / ____| |   |___ \ 
[Monitor.cpp][info]  | \  / | __ _| |    | |__   __) |
[Monitor.cpp][info]  | |\/| |/ _` | |    | '_ \ |__ < 
[Monitor.cpp][info]  | |  | | (_| | |____| | | |___) |
[Monitor.cpp][info]  |_|  |_|\__,_|\_____|_| |_|____/ 
[Monitor.cpp][info] Version: 2.3.1
[Monitor.cpp][info] ##################################
[Monitor.cpp][info] Using following CPU:


cat: write error: Broken pipe
cat: write error: Broken pipe


[Monitor.cpp][info] model name	: AMD EPYC 9454 48-Core Processor
[Monitor.cpp][info] cpu MHz		: 2750.000
[Monitor.cpp][info] Architecture:                         x86_64
[Monitor.cpp][info] L1d cache:                            1.5 MiB (48 instances)
[Monitor.cpp][info] L1i cache:                            1.5 MiB (48 instances)
[Monitor.cpp][info] L2 cache:                             48 MiB (48 instances)
[Monitor.cpp][info] L3 cache:                             256 MiB (8 instances)
[Monitor.cpp][info] Thread(s) per core:                   1
[Monitor.cpp][info] CPU(s):                               48
[Monitor.cpp][info] With available threads                8
[Monitor.cpp][info] Using following GPU:
[Monitor.cpp][info] GPU Name: NVIDIA H100 80GB HBM3
[Monitor.cpp][info] Number of GPUs: 1
[Monitor.cpp][info] Total VRAM: 81559 MB
[Monitor.cpp][info] Driver Version: 580.65.06
[Monitor.cpp][info] Currently used GPU has: 65536 threads
[Monitor.cpp][info] Currently used GPU has: 12800 K

In [4]:
# Fit settings!
FIT_TYPE='deistlerinference'
N_ROUNDS = 100
SAMPLES_PER_ROUND=300

training_settings = {}

ttree_settings = {}

FIT_ARGS = {
    "TrainingSettings": training_settings,    
}

In [5]:
# Now we can run the fit!
ui.run_fit(FIT_TYPE, N_ROUNDS, SAMPLES_PER_ROUND, {}, {})

  0%|          | 0/100 [00:00<?, ?it/s]

/scratch/henryi/venvs/.venv_sbi/lib/python3.11/site-packages/sbi/inference/trainers/npe/npe_base.py:177: UserWarning: Data x has device 'cuda:0'. Moving x to the data_device 'cuda'. Training will proceed on device 'cuda'.
  theta, x = validate_theta_and_x(
/scratch/henryi/venvs/.venv_sbi/lib/python3.11/site-packages/sbi/inference/trainers/npe/npe_base.py:177: UserWarning: Parameters theta has device 'cuda:0'. Moving theta to the data_device 'cuda'. Training will proceed on device 'cuda'.
  theta, x = validate_theta_and_x(


 Neural network successfully converged after 256 epochs.

/scratch/henryi/venvs/.venv_sbi/lib/python3.11/site-packages/sbi/utils/sbiutils.py:804: UserWarning: The passed discrete prior has no mean or stddev attribute, estimating them from samples to build affine standardizing transform.
  transform = prior_mean_std_transform(prior, device)


TypeError: cannot pickle 'pyMaCh3Tutorial._pyMaCh3Tutorial.MaCh3TutorialWrapper' object

In [ ]:


s = ui.fitter.posterior.sample((100000,), device_handler.to_tensor(ui.fitter.x0))
from matplotlib import pyplot as plt


In [ ]:
import uproot as ur

tree = ur.open("/home/henryi/sft/MaCh3Tutorial/Test.root:posteriors")
arrs = tree.arrays(library="np")


In [ ]:
plt.hist(s[:,0].cpu(), bins=100, alpha=0.3, label="ML")
# plt.hist(arrs['xsec_3'], alpha=0.3, label="MCMC")
# plt.legend()
plt.xlabel('xsec_0')
plt.show()

In [ ]:
print(s)